## Text

In [2]:
import os
import numpy as np
import re

In [3]:
class BOW():
    def __init__(self, k_val, b_val):
        # Constants
        self.k_val = k_val
        self.b_val = b_val

        # Vocabulary contains the words in the corpus and the number of documents they appear in
        self.vocabulary = {}
        self.vocabulary_arr = None
        self.vocab_size = None
        
        # Documents contains the words in each document and the number of times they appear
        self.documents = {}
        self.document_vectors = None
        self.doc_count = None
        self.max_docs = None
        self.avg_doc_length = None

        # score matrices
        self.tf_matrix = None
        self.idf_matrix = None
    
    def create(self, path_to_annotations):

        self.doc_count = 0
        for (root, dirs, files) in os.walk(path_to_annotations, topdown=True):
            print(root)
            # print(dirs)
            # print(files)
            for file in files:
                file_num = file.split('.')[0]
                file_num = int(file_num)
                # parses the annotation file and adds the words to the vocabulary
                self.parse_annotation_file(os.path.join(root, file), file_num)
                self.doc_count += 1

        self.vocab_size = len(self.vocabulary)

        self.vocabulary_arr = np.array(list(self.vocabulary))

        # make document vectors
        self.max_docs = max(self.documents.keys())+1
        self.document_vectors = np.zeros((self.max_docs, self.vocab_size))
        
        for doc_num in self.documents.keys():
            for word in self.documents[doc_num]:
                word_index = np.where(self.vocabulary_arr == word)[0][0]
                self.document_vectors[doc_num][word_index] = self.documents[doc_num][word]

        self.avg_doc_length = np.sum(self.document_vectors) / self.doc_count

    def form_matrix(self):
        if(self.vocabulary_arr is None):
            print("vocabulary is none")
            print("run create() first")

        # calculate idf
        values = self.vocabulary.values()
        values = np.array(list(values))
        self.idf_matrix = np.log((self.doc_count + 1) / (values + 0.5))

        # calculate tf
        self.tf_matrix = self.k_val*self.document_vectors / (self.k_val * (1 - self.b_val + (self.b_val * (np.sum(self.document_vectors,axis=1).T/ self.avg_doc_length))) + self.document_vectors.T).T   

        return self.tf_matrix, self.idf_matrix  

    def get_query_score(self, path_to_query, doc_num):
        if(self.idf_matrix is None):
            print("idf matrix is none")
            print("run form_matrix() first")
        if(self.tf_matrix is None):
            print("tf matrix is none")
            print("run form_matrix() first")

        vector = self.get_query_vector(path_to_query)
        tf_q = self.get_query_tf(vector)
        tf_d = self.tf_matrix[doc_num]
        idf = self.idf_matrix
        return np.sum(tf_q*tf_d*idf*idf)

    def get_query_tf(self, query_vector):
        return self.k_val*query_vector / (self.k_val * (1 - self.b_val + self.b_val * (np.sum(query_vector).T/ self.avg_doc_length)) + query_vector.T).T

    def get_query_vector(self, path_to_query):
        query_tf_vector = np.zeros(self.vocab_size)
        with open(path_to_query, 'r', encoding='latin-1') as f:
            # read the doc as a string
            doc = f.read()
            
            # get the words in the doc
            words_in_doc = self.get_words_in_doc(doc)

            for word in words_in_doc:
                word_index = np.where(self.vocabulary_arr == word)[0][0]
                query_tf_vector[word_index] = words_in_doc[word]
            
        return query_tf_vector

    def parse_annotation_file(self, path_to_annotation_file, file_number):
        # open in latin-1 encoding to avoid UnicodeDecodeError
        with open(path_to_annotation_file, 'r', encoding='latin-1') as f:
            # read the doc as a string
            doc = f.read()
            
            # get the words in the doc
            words_in_doc = self.get_words_in_doc(doc)
            
            # add the words in the document to the vocabulary
            self.documents[file_number] = words_in_doc

            for word in words_in_doc:
                if word in self.vocabulary:
                    self.vocabulary[word] += 1
                else:
                    self.vocabulary[word] = 1

    def get_words_in_doc(self, doc):

        # replace \n with space
        doc = doc.replace('\n', ' ')
        # get parts between all <TITLE> and </TITLE> tags
        titles = re.findall(r'<TITLE>(.*?)</TITLE>', doc)
        # get parts between all <DESCRIPTION> and </DESCRIPTION> tags
        descriptions = re.findall(r'<DESCRIPTION>(.*?)</DESCRIPTION>', doc)
        # get parts between all <NOTES> and </NOTES> tags
        notes = re.findall(r'<NOTES>(.*?)</NOTES>', doc)
        # get parts between all <LOCATION> and </LOCATION> tags
        locations = re.findall(r'<LOCATION>(.*?)</LOCATION>', doc)

        words_in_doc = {}

        # split into words and add to vocabulary
        for title in titles:
            for word in title.split():
                word = word.strip(' .,;:!?()[]\{\}\'\"')
                word = word.lower()
                if word not in words_in_doc:
                    words_in_doc[word] = 0
                words_in_doc[word] += 1

        for description in descriptions:
            for word in description.split():
                word = word.strip('.,;:!?()[]\{\}\'\"')
                word = word.lower()
                if word not in words_in_doc:
                    words_in_doc[word] = 0
                words_in_doc[word] += 1

        for note in notes:
            for word in note.split():
                word = word.strip('.,;:!?()[]\{\}\'\"')
                word = word.lower()
                if word not in words_in_doc:
                    words_in_doc[word] = 0
                words_in_doc[word] += 1

        for location in locations:
            for word in location.split():
                word = word.strip('.,;:!?()[]\{\}\'\"')
                word = word.lower()
                if word not in words_in_doc:
                    words_in_doc[word] = 0
                words_in_doc[word] += 1

        return words_in_doc

In [4]:
path_to_annotations = './Smol_set/annotations_complete_eng/'
k_val = 1.5
b_val = 0.75
bow = BOW(k_val, b_val)
bow.create(path_to_annotations)

./Smol_set/annotations_complete_eng/
./Smol_set/annotations_complete_eng/00
./Smol_set/annotations_complete_eng/01
./Smol_set/annotations_complete_eng/02
./Smol_set/annotations_complete_eng/03
./Smol_set/annotations_complete_eng/04
./Smol_set/annotations_complete_eng/05


In [5]:
tf_matrix, idf_matrix = bow.form_matrix()

In [6]:
sample = bow.get_query_score('./Smol_set/annotations_complete_eng/03/3489.eng',3489)

print(sample)

74.53875972166996
